# Домашняя работа №15

## Задача 1

In [1]:
import requests

In [2]:
import pandas as pd

In [3]:
from bs4 import BeautifulSoup

In [4]:
URL = 'https://habr.com/ru/all/'
KEYWORDS = ['intel', 'парсинг', 'язык', 'онлайн', 'код']
req = requests.get('https://habr.com/ru/all/')

In [5]:
soup = BeautifulSoup(req.text, 'html.parser')
soup

<!DOCTYPE html>

<html class="no-js" lang="ru">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="width=1024" name="viewport"/>
<title>Все публикации подряд / Хабр</title>
<meta content="444736788986613" property="fb:app_id">
<meta content="website" property="og:type"/>
<meta content="472597926099084" property="fb:pages"/>
<meta content="Хабр" property="og:site_name"/>
<link href="https://habr.com/images/habr_ru.png" rel="image_src"/>
<meta content="https://habr.com/images/habr_ru.png" property="og:image"/>
<meta content="1200" property="og:image:width"/>
<meta content="628" property="og:image:height"/>
<meta content="Все публикации подряд / Хабр" property="og:title"/>
<meta content="Хабр — крупнейший в Европе ресурс для IT-специалистов. Сюда приходят обсудить новости индустрии и поделиться опытом." property="og:description"/>
<meta content="71593b225aeafc4e" name="yandex-verification"/>
<meta content="unsafe-url" name="referrer"/>
<meta content

In [6]:
# news_blocks = soup.find_all('div', class_=['post__text post__text-html post__text_v1', 'post__text post__text-html post__text_v2'])
news_blocks = soup.find_all('article', class_ = 'post post_preview')
news_blocks

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/phillennium/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/c62/f71/ed6/c62f71ed62715557e8873a4b53ab9d3a.jpg" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">phillennium</span>
 </a>
 <span class="post__time">сегодня в 20:21</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/jugru/blog/522148/">Новая неделя YouTube-стримов: от Vue.js до SIMD-инструкций</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/company/jugru/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Блог компании JUG Ru Group'); }" rel="nofollow" t

In [7]:
habr_news = pd.DataFrame()
for block in news_blocks:
    post = block.find('div', class_ = ['post__text post__text-html post__text_v1',
                                        'post__text post__text-html post__text_v2'])
    post_lower = post.text.lower()
    if any([word in post_lower for word in KEYWORDS]):
        post = post.text.strip()
        date = block.find('span', class_='post__time').text
        link = block.find('a', class_ = 'btn btn_x-large btn_outline_blue post__habracut-btn').get('href')
        title_name = block.find('h2', class_ = 'post__title').text.strip()
        row = {'date': date, 'title': title_name, 'link': link, 'text': post}
        habr_news = pd.concat([habr_news, pd.DataFrame([row])])
habr_news

,date,title,link,text
0,сегодня в 19:57,Реализация расширения Active Patterns для язык...,https://habr.com/ru/company/JetBrains-educatio...,О проекте\r\nВесной 2020 года в рамках весенне...
0,сегодня в 19:00,Как использовать Google Trends для анализа клю...,https://habr.com/ru/post/522144/#habracut,"Про Google Trends слышал, наверное, каждый, кт..."
0,сегодня в 18:56,Где живут вредоносы: хакерская инфраструктура ...,https://habr.com/ru/company/trendmicro/blog/52...,Все онлайн-бизнесы нуждаются в стабильной и на...
0,сегодня в 18:26,Разработка графического профайлера Python Func...,https://habr.com/ru/company/skillfactory/blog/...,Сегодня мы делимся с вами переводом статьи соз...
0,сегодня в 17:28,Security Week 41: вредоносный код в UEFI,https://habr.com/ru/company/kaspersky/blog/522...,Эксперты «Лаборатории Касперского» опубликовал...
0,сегодня в 17:27,Serverless телеграм бот с использованием Яндек...,https://habr.com/ru/post/522126/#habracut,"Всем привет! Меня зовут Павлов Виктор, я являю..."
0,сегодня в 16:15,Взлом ESP32 путём обхода Secure Boot и Flash E...,https://habr.com/ru/company/ruvds/blog/522080/...,Мы провели исследование микроконтроллера Espre...


## Задача 2

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: - <дата утечки> - <источник утечки> - <описание утечки>

In [8]:
import json

In [9]:
email = ['xxx@xx.com', 'yyy@xx.com']
params = {'emailAddresses': email}
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64; rv:81.0) Gecko/20100101 Firefox/81.0',
          'Vaar-Header-App-Product': 'hackcheck-web-avast',
          'Vaar-Version': '0'}
req = requests.post(URL, data= json.dumps(params), headers=headers)
items = req.json()['breaches']
items

{'16604': {'site': 'avvo.com',
  'recordsCount': 4101221,
  'description': "In May 2018, the legal marketplace Avvo's database was allegedly breached. The stolen data contains usernames, passwords, and email addresses. This breach is being privately shared on the internet.",
  'publishDate': '2019-04-04T00:00:00Z'},
 '16533': {'site': 'points.com',
  'recordsCount': 383869,
  'description': 'This is a list of email addresses only, and as far as we know, the associated site itself was not compromised. We cannot currently confirm the origin or the methods used to obtain the emails, whether by website scraping or user login page enumeration. We do not believe that user passwords or other personally identifiable information (PII) was exposed, and the presence of any email address does not necessarily indicate compromise of its associated credentials. \n\nThe primary danger is an increased likelihood of being targeted by phishing or spamming campaigns.',
  'publishDate': '2019-02-21T00:00:0

In [37]:
s = pd.DataFrame(req.json()['breaches'])
df = pd.DataFrame()
df['date'] = s.loc['publishDate']
df['source'] = s.loc['site']
df['description'] = s.loc['description']
df

,date,source,description
16604,2019-04-04T00:00:00Z,avvo.com,"In May 2018, the legal marketplace Avvo's data..."
16533,2019-02-21T00:00:00Z,points.com,"This is a list of email addresses only, and as..."
16797,2019-07-04T00:00:00Z,modaoperandi.com,"In March 2019, online fashion retailer Moda Op..."
16830,2019-07-25T00:00:00Z,worldofwar.net,"At an unconfirmed date, World of Warcraft fand..."
16572,2019-03-07T00:00:00Z,armorgames.com,"In December 2018, Armor Games' database was al..."
...,...,...,...
16303,2019-03-14T00:00:00Z,jobandtalent.com,"At an unconfirmed date, Job and Talent's datab..."
16610,2019-03-28T00:00:00Z,coubic.com,"In February 2019, Coubic's database was allege..."
16859,2019-08-01T00:00:00Z,flashflashrevolution.com,"In July 2019, multiplayer video game Flash Fla..."
15,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
